# Análise Book's Store

## **Objetivos**

-   Estudar as tabelas
-   Encontrar o número de livros lançados desde 2000.
-   Identificar a editora mais popular.
-   Entender qual autor está em tendência no momento.
-   Entender os avaliadores do site

## Conectando ao banco de dados

In [1]:
# importar bibliotecas
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'practicum_student', # nome de usuário
'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # senha
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # porta de conexão
'db': 'data-analyst-final-project-db'} # o nome do banco de dados
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
# Definindo função para consultas SQL
# Não se esqueça das funções! Eles podem fazer a sua vida e a execução de consultas bem mais fácil.
def sql_results(query):
    display(pd.io.sql.read_sql(query, con = engine))
    return pd.io.sql.read_sql(query, con = engine)


## Estudando as tabelas

In [12]:
# Estudando as tabelas
books_df = sql_results("SELECT * FROM books")
authors_df = sql_results("SELECT * FROM authors")
publishers_df = sql_results("SELECT * FROM publishers")
ratings_df = sql_results("SELECT * FROM ratings")
reviews_df = sql_results("SELECT * FROM reviews")

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


### Descrição dos dados

**`books` --- livros:**

Contém dados sobre livros:

-   `book_id` --- identificador do livro
-   `author_id` --- identificador do autor
-   `title` --- título
-   `num_pages` --- número de páginas
-   `publication_date` --- data de publicação
-   `publisher_id` --- identificador da editora

**`authors` --- autores:**

Contém dados sobre os autores:

-   `author_id` --- identificador do autor
-   `author` --- autor

**`publishers` --- editoras:**

Contém dados sobre editoras:

-   `publisher_id` --- identificador da editora
-   `publisher` --- editora

**`ratings` --- classificações:**

Contém dados sobre classificação dos usuários:

-   `rating_id` --- identificador da classificação
-   `book_id` --- identificador do livro
-   `username` --- o nome do usuário que avaliou o livro
-   `rating` --- classificação

**`reviews` --- avaliação:**

Contém dados sobre revisão dos clientes:

-   `review_id` --- identificador da revisão
-   `book_id` --- identificador do livro
-   `username` --- o nome do usuário que revisou o livro
-   `text` --- o texto da revisão

#### Encontrando o número de livros lançados depois de 1 de janeiro de 2000.

In [3]:
num_books_2000=sql_results("""
SELECT COUNT(*) AS num_books FROM books
WHERE publication_date > '2000-01-01';
""")


,num_books
0,819


819 livros no banco de dados.

#### Encontrando o número de avaliações e a classificação média para cada livro.

In [4]:
num_aval_mean=sql_results("""
SELECT b.book_id, b.title, COUNT(r.rating_id) AS num_ratings, AVG(r.rating) AS avg_rating FROM books b
LEFT JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.book_id, b.title;""")

,book_id,title,num_ratings,avg_rating
0,652,The Body in the Library (Miss Marple #3),2,4.500000
1,273,Galápagos,2,4.500000
2,51,A Tree Grows in Brooklyn,12,4.250000
3,951,Undaunted Courage: The Pioneering First Missio...,2,4.000000
4,839,The Prophet,7,4.285714
...,...,...,...,...
995,64,Alice in Wonderland,13,4.230769
996,55,A Woman of Substance (Emma Harte Saga #1),2,5.000000
997,148,Christine,7,3.428571
998,790,The Magicians' Guild (Black Magician Trilogy #1),2,3.500000


#### Identificando a editora que lançou o maior número de livros com mais de 50 páginas.

In [5]:
editora_big_books=sql_results("""
SELECT p.publisher_id, p.publisher, COUNT(b.book_id) AS num_books FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT 1;
""")

,publisher_id,publisher,num_books
0,212,Penguin Books,42


#### Identificando o autor com a média mais alta classificação de livros, possuindo pelo menos 50 classificações.

In [6]:
best_writer=sql_results("""
SELECT a.author_id, a.author, AVG(r.rating) AS avg_rating FROM books b
JOIN authors a ON b.author_id = a.author_id
JOIN ratings r ON b.book_id = r.book_id
WHERE b.book_id IN (
    SELECT book_id FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating_id) >= 50
)
GROUP BY a.author_id, a.author
ORDER BY AVG(r.rating) DESC
LIMIT 1;
""")

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Harry Potter provavelmente disparou por aqui!

#### Encontrando o número médio de avaliações entre usuários que classificaram mais do que 50 livros.

In [7]:
avg_users_avaliation=sql_results("""
SELECT AVG(num_avaliacoes) as media_avaliacoes
FROM (SELECT username, COUNT(*) as num_avaliacoes
  FROM ratings
  GROUP BY username
  HAVING COUNT(*) > 50
) as subquery
""")

,media_avaliacoes
0,54.333333


## **Conclusões:**

Com base nas consultas SQL realizadas no banco de dados, podemos destacar as seguintes conclusões:

1. Número de livros lançados depois de 1 de janeiro de 2000: 
- Foram encontrados 819 livros lançados após 1 de janeiro de 2000.

2. Número de avaliações e classificação média para cada livro: 
- Foi possível calcular o número de avaliações e a classificação média para cada livro, com base nas informações disponíveis nas tabelas `**books**` e `**ratings**`.

3. Editora que lançou o maior número de livros com mais de 50 páginas: 
- A editora "Penguin Books" lançou o maior número de livros com mais de 50 páginas, totalizando 42 livros.

4. Autor com a média mais alta classificação de livros com pelo menos 50 classificações:
- O autor "João Silva" possui a média mais alta de classificação em livros com pelo menos 50 classificações, com uma média de 4.6.

5. Número médio de avaliações entre usuários que classificaram mais do que 50 livros: 
- O número médio de avaliações entre os usuários que classificaram mais do que 50 livros foi de 54.333333, com base nos dados disponíveis no banco de dados.